# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp app

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
except: 
    from load_data import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import json
from git import Repo
import pandas as pd
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, html, Patch, ctx  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import random

## Load Data

In [ ]:
#| export
sol_geo = SolomonGeo.load_pickle("/testData/")
geo_df = sol_geo.geo_df

In [ ]:
# TODO I should build figures and maps in another script

#| export
def define_map(sol_df:SolomonGeo # Solomon geo object containing census data to input into map
                )->type(go.Figure()): # Returns a graph object figure
    '''
    Creates and returns the base cloreopath map
    '''
    # TODO - should I update this into a class with methods for updating
    # the other things? Acutally maybe as another function if the update is done through patch
    
    # cols_dd dictates the aggregation that will be visable
    cols_dd = sol_df.geo_levels
    # we need to add this to select which trace 
    # is going to be visible
    visible = np.array(cols_dd)
    # define traces and buttons at once
    traces = []
    buttons = []
    # TODO if fails remember I changed visible from cols_dd
    for value in cols_dd:
        traces.append(go.Choroplethmapbox(
                                geojson=sol_df.get_geojson(agg_filter = value),
                               locations=sol_df.get_df(agg_filter = value).index,
                               z = sol_df.get_df(agg_filter = value)['total_pop'],
                               colorscale="deep",
                                marker_line_width = 0.5,
                                zauto=True,
                visible= True if value==cols_dd[0] else False))
        
    # Show figure
    fig = go.Figure(data=traces)
    # This is in order to get the first title displayed correctly
    first_title = cols_dd[0]
    fig.update_layout(title=f"<b>{first_title}</b>",
                        title_x=0.5,
                        mapbox_style = 'carto-positron',
                        mapbox_zoom = 5,
                        mapbox_center={"lat": -9.565766, "lon": 162.012453},
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return fig


In [ ]:
show_doc(define_map)

---

### define_map

>      define_map (sol_df:SolomonIslandsDataMap.load_data.SolomonGeo)

Creates and returns the base cloreopath map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| sol_df | SolomonGeo | Solomon geo object containing census data to input into map |
| **Returns** | **Figure** | **Returns a graph object figure** |

In [ ]:
fig = define_map(sol_geo)

In [ ]:
fig['data'][0]['visible']

True

## Setup Dash Components

In [ ]:
#| export
# Build your components
# FYI the best themes seem to be: [Darkly, Flatly, Minty, Slate, JOURNAL]
app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
server = app.server
load_figure_template("minty")

mytitle = dcc.Markdown(children='')
mygraph = dcc.Graph(figure=define_map(sol_geo))
geos = geo_df.loc[:, 'agg'].unique()
cen_vars = sol_geo.census_vars
#dropdown_geo = dbc.Dropdown(options=geos,
#                        value=geos[0],  # initial value displayed when page first loads
#                        clearable=False)
button_group = html.Div(
    [
        dbc.RadioItems(
            id="radio_geo",
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-primary",
            labelCheckedClassName="active",
            
            options=[
                {"label": geos[0], "value": geos[0]},
                {"label": geos[1], "value": geos[1]},
                {"label": geos[2], "value": geos[2]},
            ],
            value=geos[0],
            #vertical = True, # TODO work this out
        ),
        html.Div(id="output"),
        
    ],
    className="radio-group",
    
)
dropdown_var = dcc.Dropdown(options=cen_vars,
                        value=cen_vars[-1],  # initial value displayed when page first loads
                        clearable=False)

navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Census Data", href="#")),
        dbc.DropdownMenu(
            children=[
                dbc.DropdownMenuItem("More pages coming soon", header=True),
                dbc.DropdownMenuItem("Population Projection", href="#"),
            ],
            nav=True,
            in_navbar=True,
            label="More",
        ),
    ],
    brand="Solomon Islands Data Map",
    brand_href="#",
    color="primary",
    dark=True,
)

In [ ]:
geos[0]

'ward'

Create a starting map

## Create a sidebar
The sidebar will 

In [ ]:
#| export
# Note, for now I am not using a sidebar style as I do not want to fix the width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "24rem",
    "padding": "2rem 1rem",
    #"background-color": "#f8f9fa",
}


sidebar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        dbc.Nav(
            [
                html.P("Geography"), # TODO add a little info button here with link to geo explanation
                button_group,
                html.Br(),
                html.P("Data"), # TODO add a little info button here with link to geo explanation
                dropdown_var,
                #html.Br(),
                #dcc.Dropdown(id = 'three')

            ],
            vertical=True,
            pills=True,
        ),
    ],
    #style=SIDEBAR_STYLE,
)



## Customize the Layout

In [ ]:
#| export
app.layout = dbc.Container([
                dbc.Row([
                    navbar
                ]),
                dbc.Row([
                    dbc.Col([{}], width = 3),

                    dbc.Col([mytitle],width = 9, style = {'margin-left':'7px','margin-top':'7px'})
                    ]),
                dbc.Row(
                    [dbc.Col(sidebar, width = 3),
                    dbc.Col([mygraph,
                            dbc.Row(
                                []
                            )
                            ], width = 9)#, style = {'margin-left':'15px', 'margin-top':'7px', 'margin-right':'15px'})
                     ], justify = 'center'),                    
                ], fluid = True)

## Build callbacks to create interaction

In [ ]:
#| export
# Callback allows components to interact

# TODO put title in it's own callback
@app.callback(
    Output(mygraph, 'figure'),
    Output(mytitle, 'children'),
    Input(button_group, 'value'),
    Input(dropdown_var, 'value'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_geography(geo_input:str, # User input from the geography dropdown
                     census_var:str # User input for the census variable
              )->(type(go.Figure()), str): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    patched_figure = Patch()
    button_clicked = ctx.triggered_id

    if button_clicked == radio_geo.id:
        # Update disaplayed geography based on 
        for geo in sol_geo.geo_levels:
            i = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            patched_figure['data'][i]['visible'] = geo_input == geo
            print(geo_input == geo)
    elif button_clicked == dropdown_var.id:
        # Update the z values in map to the data for the requested
        # census variable
        
        for geo in sol_geo.geo_levels:
        # Ar updates the z value ie. data disaplyed each time
        # TODO this is fairly inefficient, as we are processing each time
        # Maybe faster framework like polars could help? or caching but would require a lot of caching
            i = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            ar = sol_geo.get_df(agg_filter = geo, var_filter = census_var).values
            ar = ar.reshape((ar.shape[0],))
            patched_figure['data'][i]['z'] = ar

    # returned objects are assigned to the component property of the Output
    return patched_figure, '# Solomon Islands Data map - ' + geo_input

In [ ]:
show_doc(update_geography)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/app.py#L160){target="_blank" style="float:right; font-size:smaller"}

### update_geography

>      update_geography (geo_input:str, census_var:str)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| census_var | str | User input for the census variable |
| **Returns** | **(<class 'plotly.graph_objs._figure.Figure'>, <class 'str'>)** | **Returns a graph object figure after being updated and the dynamic title** |

In [ ]:
fig['data'][2]['z'].shape

(10,)

In [ ]:
# TODO Do the title in one callback that listens for changes to either of them

In [ ]:
ar = sol_geo.get_df(agg_filter = 'province', var_filter = 'female_pop').values
ar = ar.reshape((ar.shape[0],))
ar.shape

(10,)

In [ ]:
#patched_figure = Patch()
user_input = 'constituency'
for geo in sol_geo.geo_levels:
    i = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
    fig['data'][i]['visible'] = user_input == geo
    print(user_input == geo)

False
True
False


In [ ]:
# TODO undo hardcoding
test_eq(fig['data'][0]['visible'], False)
test_eq(fig['data'][1]['visible'], True)
test_eq(fig['data'][2]['visible'], False)

### Update the colour columns based on data selection

In [ ]:
# TODO the above demonstrates that we can stack mutliple graphs and toggle the visibility and potentially z value in order to 
# quickly update our graph

fig = px.choropleth_mapbox(sol_geo.get_df(agg_filter = 'ward'),
        # TODO - may need to use old version of mapping to get visibility hack to work..
                        #geojson = str(repo.working_tree_dir) + "/assets/sol_geo.json",
                            geojson=sol_geo.get_geojson(agg_filter = 'ward'),
                           locations=sol_geo.get_df(agg_filter = 'ward').index,
                               color = 'total_pop',
                           color_continuous_scale="deep",
                               mapbox_style = 'carto-positron',
                               opacity = 0.8,
                        zoom = 5,
                        center={"lat": -9.565766, "lon": 162.012453})
print(fig["data"][0]["z"])

## Run the dash app

In [ ]:
#| export
#| server
# Run app
if __name__=='__main__':
    app.run_server(debug=True, port = random.randint(1000, 9999)) # Random int mitigates port collision

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()